In [ ]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

In [ ]:
cd(@__DIR__)
isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

In [ ]:
io = open("input.txt", "r")
text = collect(read(io, String))

* In the above cell:
  1. [read(io, String)](https://docs.julialang.org/en/v1/base/io-network/#Base.read) will read the whole context of the input file as a String.
  1. [collect](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,Any}) is a JPL's built-in function.

In [ ]:
alphabet = [unique(text)..., '_']

* The syntax [[]](https://docs.julialang.org/en/v1/manual/functions/#Operators-With-Special-Names-1) is `hcat`.
  * In, JPL [operators are functions](https://docs.julialang.org/en/v1/manual/functions/#Operators-Are-Functions-1). One can apply them using parenthesized argument lists, just as you would any other function.

In [ ]:
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

* `ch -> onehot(ch, alphabet)` is JPL's [anonymous functions](https://docs.julialang.org/en/v0.6.1/manual/functions/#man-anonymous-functions-1).
* `onehot` is a function implemented in Flux. It converts a word into its index in the dictionary.

In [ ]:
N = length(alphabet)

seqlen = 50
nbatch = 50

Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

* [partition](https://docs.julialang.org/en/v1/base/iterators/#Base.Iterators.partition) is a built-in function in JPL. It iterates over a collection _n_ elements at a time.

In [ ]:
m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)
m = gpu(m)

function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m), 0.01)
tx, ty = (gpu.(Xs[5]), gpu.(Ys[5]))
evalcb = () -> @show loss(tx, ty)

Flux.train!(loss, zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))

# Sampling
m = cpu(m)

function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println

# evalcb = function ()
#   @show loss(Xs[5], Ys[5])
#   println(sample(deepcopy(m), alphabet, 100))
# end